# NOTEBOOK GOAL: STOCK FORECASTING WITH GUI
### Train an AI model to predict the market price of a stock.
### Client use a GUI for insert parameters

In [ ]:
# Import library
import tkinter as tk
from tkinter import ttk, messagebox
import yfinance as yf
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
from prophet import Prophet
import pandas as pd

In [ ]:
# Function to display data in a new windows
def display_data(data, title):
    window = tk.Toplevel()
    window.title(title)
    text = tk.Text(window, wrap='none')
    text.insert(tk.END, data)
    text.pack(expand=True, fill='both')

In [ ]:
# Function to retrieve and view historical market data
def show_hist_data():
    ticker = ticker_entry.get()
    start_date = start_entry.get()
    end_date = end_entry.get()
    if not ticker or not start_date or not end_date:
        messagebox.showerror("Error", "Please enter all fields (Ticker, Start Date, End Date).")
        return
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date)
        display_data(hist.to_string(), "Historical Market Data")
        plot_data(hist, ticker)
    except Exception as e:
        messagebox.showerror("Error", str(e))

In [ ]:
# Function to retrieve and view balances
def show_financial_data(data_type):
    ticker = ticker_entry.get()
    if not ticker:
        messagebox.showerror("Error", "Please enter the Ticker.")
        return
    try:
        stock = yf.Ticker(ticker)
        if data_type == "income_stmt":
            data = stock.income_stmt
        elif data_type == "balance_sheet":
            data = stock.balance_sheet
        elif data_type == "cashflow":
            data = stock.cashflow
        display_data(data.to_string(), data_type.replace("_", " ").title())
    except Exception as e:
        messagebox.showerror("Error", str(e))

In [ ]:
# Function to track historical data
def plot_data(data, ticker):
    fig, ax = plt.subplots()
    data['Close'].plot(ax=ax, title=f'Historical Closing Prices of {ticker}')
    ax.set_xlabel("Date")
    ax.set_ylabel("Closing Price")

    window = tk.Toplevel()
    window.title(f'Historical Data Graph of {ticker}')
    canvas = FigureCanvasTkAgg(fig, master=window)
    canvas.draw()
    
    # Add toolbar for zoom and values
    toolbar = NavigationToolbar2Tk(canvas, window)
    toolbar.update()
    canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

In [ ]:
# Feature to predict stock prices using Prophet
def forecast_data():
    ticker = ticker_entry.get()
    start_date = start_entry.get()
    end_date = end_entry.get()
    forecast_end_date = forecast_end_entry.get()
    if not ticker or not start_date or not end_date or not forecast_end_date:
        messagebox.showerror("Error", "Please enter all fields (Ticker, Start Date, End Date, Forecast End Date).")
        return
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date)
        df = hist[['Close']].reset_index()
        df['Date'] = df['Date'].dt.tz_localize(None)  # Remove timezone
        df.rename(columns={'Date': 'ds', 'Close': 'y'}, inplace=True)

        m = Prophet()
        m.fit(df)
        future = m.make_future_dataframe(periods=(datetime.strptime(forecast_end_date, '%Y-%m-%d') - datetime.strptime(end_date, '%Y-%m-%d')).days)
        forecast = m.predict(future)

        fig1 = m.plot(forecast)
        fig2 = m.plot_components(forecast)

        display_forecast(fig1, ticker)
        display_forecast(fig2, ticker)
    except Exception as e:
        messagebox.showerror("Error", str(e))

In [ ]:
# Function to view forecast graphs
def display_forecast(fig, ticker):
    window = tk.Toplevel()
    window.title(f'Prediction of {ticker}')
    canvas = FigureCanvasTkAgg(fig, master=window)
    canvas.draw()
    
    # Add toolbar for zoom and values
    toolbar = NavigationToolbar2Tk(canvas, window)
    toolbar.update()
    canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

In [ ]:
# Function to retrieve and view news
def show_news():
    ticker = ticker_entry.get()
    if not ticker:
        messagebox.showerror("Error", "Please enter the Ticker.")
        return
    try:
        stock = yf.Ticker(ticker)
        news = stock.news
        news_str = "\n\n".join([f"Title: {item['title']}\nLink: {item['link']}\n" for item in news])
        display_data(news_str, "Recent news")
    except Exception as e:
        messagebox.showerror("Error", str(e))

In [ ]:
# GUI's main window 
root = tk.Tk()
root.title("Stock Information")

# Labels and input fields for tickers and date range
ttk.Label(root, text="Ticker:").grid(row=0, column=0, padx=5, pady=5)
ticker_entry = ttk.Entry(root)
ticker_entry.grid(row=0, column=1, padx=5, pady=5)

ttk.Label(root, text="Start Date (YYYY-MM-DD):").grid(row=1, column=0, padx=5, pady=5)
start_entry = ttk.Entry(root)
start_entry.grid(row=1, column=1, padx=5, pady=5)

ttk.Label(root, text="End Date (YYYY-MM-DD):").grid(row=2, column=0, padx=5, pady=5)
end_entry = ttk.Entry(root)
end_entry.grid(row=2, column=1, padx=5, pady=5)

ttk.Label(root, text="Forecast End Date (YYYY-MM-DD):").grid(row=3, column=0, padx=5, pady=5)
forecast_end_entry = ttk.Entry(root)
forecast_end_entry.grid(row=3, column=1, padx=5, pady=5)

# Crea pulsanti per ciascuna categoria di dati
ttk.Button(root, text="Show Historical Market Data", command=show_hist_data).grid(row=4, column=0, columnspan=2, pady=5, padx=10, sticky='ew')
ttk.Button(root, text="Show Balance", command=lambda: show_financial_data("income_stmt")).grid(row=5, column=0, columnspan=2, pady=5, padx=10, sticky='ew')
ttk.Button(root, text="Show Balance Sheet", command=lambda: show_financial_data("balance_sheet")).grid(row=6, column=0, columnspan=2, pady=5, padx=10, sticky='ew')
ttk.Button(root, text="Show Cash Flow", command=lambda: show_financial_data("cashflow")).grid(row=7, column=0, columnspan=2, pady=5, padx=10, sticky='ew')
ttk.Button(root, text="Predict Stock Prices", command=forecast_data).grid(row=8, column=0, columnspan=2, pady=5, padx=10, sticky='ew')
ttk.Button(root, text="Show news", command=show_news).grid(row=9, column=0, columnspan=2, pady=5, padx=10, sticky='ew')

root.mainloop()